In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import statsmodels.api as sm
import pylab as py
import scipy.stats as stats 
from scipy.stats import norm, skew, probplot
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, RidgeCV, LassoCV, ElasticNetCV
from sklearn.decomposition import PCA

In [2]:
from Feat_eng import Y_train, X_train, Y_test, X_test, c1_train, c2_train, c3_train, c1_test, c2_test, c3_test, Y_c1train, Y_c2train, Y_c3train, Y_c1test, Y_c2test ,Y_c3test  

/Users/Lena/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/Lena/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/Users/Lena/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

## Run a Lasso Regression

In [3]:
# Run a Lasso Regression on all variables

scaler = StandardScaler().fit(X_train)
features = scaler.transform(X_train)
X_std = pd.DataFrame(features, columns = X_train.columns)

alphas = 10**np.linspace(10,-2,500)*0.5

lassocv = LassoCV(alphas = alphas, random_state=0)
lassocv.fit(X_std, Y_train)
lassocv_alpha = lassocv.alpha_

las = Lasso(alpha = lassocv.alpha_, random_state = 0)
las.fit(X_std,Y_train)

print('Score for training data:', las.score(X_std,Y_train))

pred = las.predict(X_std)
pred_unlog = np.exp(pred)
Y_unlog = np.exp(Y_train)
train_RMSE = mean_squared_error(Y_unlog, pred_unlog, squared = False)
print('(not log) RMSE for training data is:', train_RMSE)

test_features = scaler.transform(X_test)
X_test_std = pd.DataFrame(test_features, columns = X_test.columns)

print('Score for test data:', las.score(X_test_std,Y_test))

pred_test = las.predict(X_test_std)
pred_test_unlog = np.exp(pred_test)
Y_test_unlog = np.exp(Y_test)
Test_RMSE = mean_squared_error(Y_test_unlog, pred_test_unlog, squared = False)
print('(not log) RMSE for test data is:', Test_RMSE)

las_imp = pd.DataFrame()

las_imp['Feature'] = X_test_std.columns
las_imp['Importance'] = list(las.coef_)
las_imp['Abs Importance'] = abs(las_imp['Importance'])
las_imp = las_imp.sort_values(by = 'Abs Importance', ascending = False)
las_imp

ranking = list(range(1,las_imp.shape[0]+1))
las_imp['ranking'] = ranking
las_imp

/Users/Lena/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Score for training data: 0.9879606154605388
(not log) RMSE for training data is: 76.62380735151801
Score for test data: 0.9869120117474273
(not log) RMSE for test data is: 83.51017972195741


,Feature,Importance,Abs Importance,ranking
141,log_value,0.409724,0.409724,1
139,interest,-0.008158,0.008158,2
46,owner_occupied_housing_units_lower_value_quartile,0.003130,0.003130,3
138,gdp,0.002230,0.002230,4
21,vacant_housing_units,-0.000556,0.000556,5
...,...,...,...,...
52,dwellings_1_units_detached,-0.000000,0.000000,148
53,dwellings_1_units_attached,-0.000000,0.000000,149
54,dwellings_2_units,-0.000000,0.000000,150
55,dwellings_3_to_4_units,-0.000000,0.000000,151


In [4]:
r2_train = las.score(X_std, Y_train)
r2_test = las.score(X_test_std, Y_test)
lasso_output = pd.DataFrame()
lasso_output['all'] = [train_RMSE, Test_RMSE, r2_train, r2_test]
lasso_output.index = ['RMSE_train', 'RMSE_test', 'R2_train', 'R2_test']
lasso_output

,all
RMSE_train,76.623807
RMSE_test,83.510180
R2_train,0.987961
R2_test,0.986912


## Run a PCA 

In [5]:
scaler_pca = StandardScaler().fit(X_train)
features = scaler_pca.transform(X_train)
df_scal = pd.DataFrame(features, columns = X_train.columns)

In [6]:
pca = PCA(n_components = 0.95)
df_reduced = pca.fit_transform(df_scal)

print(df_reduced.shape)

df_reduced = pd.DataFrame(pca.components_, columns = X_train.columns, index = list(range(1,df_reduced.shape[1]+1)))
df_reduced = df_reduced.apply(lambda x: abs(x))


(5122, 44)


In [7]:
# Try to reduce features by finding most important one for each of the dimensions
feat = []
for i in range(0,df_reduced.shape[0]):
    ff = df_reduced.iloc[i].idxmax()
    df_reduced.drop(ff, inplace = True, axis = 1)
    feat.append(ff)

In [8]:
feat.append('log_value')
feat

['total_pop',
 'income_per_capita',
 'dwellings_1_units_detached',
 'vacant_housing_units_for_rent',
 'hispanic_ratio',
 'cpi',
 'year',
 'vacant_housing_units',
 'median_age',
 'median_year_structure_built',
 'two_parents_mother_in_labor_force_families_with_young_children',
 'amerindian_ratio',
 'amerindian_including_hispanic',
 'commute_5_9_mins',
 'dwellings_3_to_4_units',
 'armed_forces',
 'gdp',
 'interest',
 'dwellings_1_units_attached',
 'other_race_ratio',
 'two_parents_not_in_labor_force_families_with_young_children',
 'employed_agriculture_forestry_fishing_hunting_mining',
 'mobile_homes',
 'female_female_households',
 'housing_built_2005_or_later',
 'commuters_by_bus',
 'male_male_households',
 'male_ratio',
 'est',
 'white_ratio',
 'million_dollar_housing_units',
 'housing_built_2000_to_2004',
 'vacant_housing_units_for_sale',
 'gini_index',
 'dwellings_5_to_9_units',
 'different_house_year_ago_same_city',
 'father_in_labor_force_one_parent_families_with_young_children',
 '

In [9]:
X_red_train = X_train[feat]
X_red_test = X_test[feat]

In [10]:
# Run Lasso Regression on PCA variables:
scaler_red = StandardScaler().fit(X_red_train)
features_red = scaler_red.transform(X_red_train)
X_red_std = pd.DataFrame(features_red, columns = X_red_train.columns)

alphas = 10**np.linspace(10,-2,500)*0.5

lassocv_red = LassoCV(alphas = alphas, random_state=0)
lassocv_red.fit(X_red_std, Y_train)
lassocv_red_alpha = lassocv_red.alpha_

las_red = Lasso(alpha = lassocv_red.alpha_, random_state = 0)
las_red.fit(X_red_std,Y_train)

print('Score for training data:', las_red.score(X_red_std,Y_train))

pred_red = las_red.predict(X_red_std)
pred_red = np.exp(pred_red)
Y_train_red = np.exp(Y_train)
train_RMSE_red = mean_squared_error(Y_train_red, pred_red, squared = False)
print('(not log) RMSE for training data is:', train_RMSE_red)

test_features_red = scaler_red.transform(X_red_test)
X_red_test_std = pd.DataFrame(test_features_red, columns = X_red_test.columns)

print('Score for test data:', las_red.score(X_red_test_std,Y_test))

pred_test_red = las_red.predict(X_red_test_std)
pred_test_red = np.exp(pred_test_red)
Y_test_red = np.exp(Y_test)
Test_RMSE_red = mean_squared_error(Y_test_red, pred_test_red, squared = False)
print('(not log) RMSE for test data is:', Test_RMSE_red)

las_imp_red = pd.DataFrame()

las_imp_red['Feature'] = X_red_test_std.columns
las_imp_red['Importance'] = list(las_red.coef_)
las_imp_red['Abs Importance'] = abs(las_imp_red['Importance'])
las_imp_red = las_imp_red.sort_values(by = 'Abs Importance', ascending = False)
las_imp_red

ranking_red = list(range(1,las_imp_red.shape[0]+1))
las_imp_red['ranking'] = ranking_red
las_imp_red

/Users/Lena/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Score for training data: 0.9879162010630738
(not log) RMSE for training data is: 76.57681297586684
Score for test data: 0.986287999468927
(not log) RMSE for test data is: 86.45155974668727


,Feature,Importance,Abs Importance,ranking
44,log_value,0.412199,0.412199,1
17,interest,-0.008392,0.008392,2
16,gdp,0.002036,0.002036,3
7,vacant_housing_units,-0.000746,0.000746,4
33,gini_index,-0.000000,0.000000,5
26,male_male_households,0.000000,0.000000,6
27,male_ratio,0.000000,0.000000,7
28,est,0.000000,0.000000,8
29,white_ratio,0.000000,0.000000,9
30,million_dollar_housing_units,0.000000,0.000000,10


In [11]:
r2_train = las_red.score(X_red_std,Y_train)
r2_test = las_red.score(X_red_test_std, Y_test)
lasso_output['all_PCA'] = [train_RMSE_red, Test_RMSE_red, r2_train, r2_test]
lasso_output

,all,all_PCA
RMSE_train,76.623807,76.576813
RMSE_test,83.510180,86.451560
R2_train,0.987961,0.987916
R2_test,0.986912,0.986288


## Run a Lasso Regression on the Clusters

In [12]:
# Run Lasso Regression on Cluster 1:
scaler = StandardScaler().fit(c1_train)
features = scaler.transform(c1_train)
c1_strain = pd.DataFrame(features, columns = c1_train.columns)

alphas = 10**np.linspace(10,-2,500)*0.5

lassocv = LassoCV(alphas = alphas, random_state=0)
lassocv.fit(c1_strain, Y_c1train)
lassocv_alpha = lassocv.alpha_

las = Lasso(alpha = lassocv.alpha_, random_state = 0)
las.fit(c1_strain,Y_c1train)

print('Score for training data:', las.score(c1_strain,Y_c1train))

pred = las.predict(c1_strain)
pred = np.exp(pred)
Y_c1_train = np.exp(Y_c1train)
train_RMSE = mean_squared_error(Y_c1_train, pred, squared = False)
print('(not log) RMSE for training data is:', train_RMSE)

test_features = scaler.transform(c1_test)
c1_stest = pd.DataFrame(test_features, columns = c1_test.columns)

print('Score for test data:', las.score(c1_stest,Y_c1test))

pred_test = las.predict(c1_stest)
pred_test = np.exp(pred_test)
Y_c1_test = np.exp(Y_c1test)
Test_RMSE = mean_squared_error(Y_c1_test, pred_test, squared = False)
print('(not log) RMSE for test data is:', Test_RMSE)

las_imp = pd.DataFrame()

las_imp['Feature'] = c1_strain.columns
las_imp['Importance'] = list(las.coef_)
las_imp['Abs Importance'] = abs(las_imp['Importance'])
las_imp = las_imp.sort_values(by = 'Abs Importance', ascending = False)
las_imp

ranking = list(range(1,las_imp.shape[0]+1))
las_imp['ranking'] = ranking
las_imp

/Users/Lena/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Score for training data: 0.9856599044363238
(not log) RMSE for training data is: 90.7300513864488
Score for test data: 0.9658948699529601
(not log) RMSE for test data is: 139.7065601411135


,Feature,Importance,Abs Importance,ranking
141,log_value,0.337493,0.337493,1
139,interest,-0.012278,0.012278,2
46,owner_occupied_housing_units_lower_value_quartile,0.002828,0.002828,3
95,employed_public_administration,-0.002810,0.002810,4
63,housing_built_1939_or_earlier,0.002521,0.002521,5
...,...,...,...,...
52,dwellings_1_units_detached,-0.000000,0.000000,148
53,dwellings_1_units_attached,-0.000000,0.000000,149
54,dwellings_2_units,0.000000,0.000000,150
55,dwellings_3_to_4_units,0.000000,0.000000,151


In [13]:
r2_train = las.score(c1_strain,Y_c1train)
r2_test = las.score(c1_stest,Y_c1test)
lasso_output['Clust_1'] = [train_RMSE, Test_RMSE, r2_train, r2_test]
lasso_output

,all,all_PCA,Clust_1
RMSE_train,76.623807,76.576813,90.730051
RMSE_test,83.510180,86.451560,139.706560
R2_train,0.987961,0.987916,0.985660
R2_test,0.986912,0.986288,0.965895


In [14]:
# Run a PCA on Cluster 1:
scaler_pca = StandardScaler().fit(c1_train)
features = scaler_pca.transform(c1_train)
df1_scal = pd.DataFrame(features, columns = c1_train.columns)

pca = PCA(n_components = 0.95)
df_reduced = pca.fit_transform(df1_scal)

df_reduced = pd.DataFrame(pca.components_, columns = c1_train.columns, index = list(range(1,df_reduced.shape[1]+1)))
df_reduced = df_reduced.apply(lambda x: abs(x))

feat1 = []
for i in range(0,df_reduced.shape[0]):
    ff = df_reduced.iloc[i].idxmax()
    df_reduced.drop(ff, inplace = True, axis = 1)
    feat1.append(ff)

feat1.append('log_value')

c1_train = c1_train[feat1]
c1_test = c1_test[feat1]

# Run Lasso Regression on Cluster 1 with PCA variables:

scaler = StandardScaler().fit(c1_train)
features = scaler.transform(c1_train)
c1_strain = pd.DataFrame(features, columns = c1_train.columns)

alphas = 10**np.linspace(10,-2,500)*0.5

lassocv = LassoCV(alphas = alphas, random_state=0)
lassocv.fit(c1_strain, Y_c1train)
lassocv_alpha = lassocv.alpha_

las = Lasso(alpha = lassocv.alpha_, random_state = 0)
las.fit(c1_strain,Y_c1train)

print('Score for training data:', las.score(c1_strain,Y_c1train))

pred = las.predict(c1_strain)
pred = np.exp(pred)
Y_c1_train = np.exp(Y_c1train)
train_RMSE = mean_squared_error(Y_c1_train, pred, squared = False)
print('(not log) RMSE for training data is:', train_RMSE)

test_features = scaler.transform(c1_test)
c1_stest = pd.DataFrame(test_features, columns = c1_test.columns)

print('Score for test data:', las.score(c1_stest,Y_c1test))

pred_test = las.predict(c1_stest)
pred_test = np.exp(pred_test)
Y_c1_test = np.exp(Y_c1test)
Test_RMSE = mean_squared_error(Y_c1_test, pred_test, squared = False)
print('(not log) RMSE for test data is:', Test_RMSE)

las_imp = pd.DataFrame()

las_imp['Feature'] = c1_strain.columns
las_imp['Importance'] = list(las.coef_)
las_imp['Abs Importance'] = abs(las_imp['Importance'])
las_imp = las_imp.sort_values(by = 'Abs Importance', ascending = False)
las_imp

ranking = list(range(1,las_imp.shape[0]+1))
las_imp['ranking'] = ranking
las_imp

/Users/Lena/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Score for training data: 0.9851991391818953
(not log) RMSE for training data is: 93.02280663277647
Score for test data: 0.9872613474022446
(not log) RMSE for test data is: 79.81914700263442


,Feature,Importance,Abs Importance,ranking
41,log_value,0.341301,0.341301,1
7,cpi,-0.012731,0.012731,2
5,employed_public_administration,-0.003123,0.003123,3
15,gdp,0.001560,0.001560,4
11,asian_ratio,0.000238,0.000238,5
29,median_age,0.000034,0.000034,6
38,dwellings_1_units_attached,-0.000000,0.000000,7
37,dwellings_5_to_9_units,0.000000,0.000000,8
25,housing_built_2005_or_later,-0.000000,0.000000,9
26,employed_wholesale_trade,0.000000,0.000000,10


In [15]:
r2_train = las.score(c1_strain,Y_c1train)
r2_test = las.score(c1_stest,Y_c1test)
lasso_output['Clust_1_PCA'] = [train_RMSE, Test_RMSE, r2_train, r2_test]
lasso_output

,all,all_PCA,Clust_1,Clust_1_PCA
RMSE_train,76.623807,76.576813,90.730051,93.022807
RMSE_test,83.510180,86.451560,139.706560,79.819147
R2_train,0.987961,0.987916,0.985660,0.985199
R2_test,0.986912,0.986288,0.965895,0.987261


In [16]:
# Run Lasso Regression on Cluster 2:

scaler = StandardScaler().fit(c2_train)
features = scaler.transform(c2_train)
c2_strain = pd.DataFrame(features, columns = c2_train.columns)

alphas = 10**np.linspace(10,-2,500)*0.5

lassocv = LassoCV(alphas = alphas, random_state=0)
lassocv.fit(c2_strain, Y_c2train)
lassocv_alpha = lassocv.alpha_

las = Lasso(alpha = lassocv.alpha_, random_state = 0)
las.fit(c2_strain,Y_c2train)

print('Score for training data:', las.score(c2_strain,Y_c2train))

pred = las.predict(c2_strain)
pred = np.exp(pred)
Y_c2_train = np.exp(Y_c2train)
train_RMSE = mean_squared_error(Y_c2_train, pred, squared = False)
print('(not log) RMSE for training data is:', train_RMSE)

test_features = scaler.transform(c2_test)
c2_stest = pd.DataFrame(test_features, columns = c2_test.columns)

print('Score for test data:', las.score(c2_stest,Y_c2test))

pred_test = las.predict(c2_stest)
pred_test = np.exp(pred_test)
Y_c2_test = np.exp(Y_c2test)
Test_RMSE = mean_squared_error(Y_c2_test, pred_test, squared = False)
print('(not log) RMSE for test data is:', Test_RMSE)

las_imp = pd.DataFrame()

las_imp['Feature'] = c2_strain.columns
las_imp['Importance'] = list(las.coef_)
las_imp['Abs Importance'] = abs(las_imp['Importance'])
las_imp = las_imp.sort_values(by = 'Abs Importance', ascending = False)
las_imp

ranking = list(range(1,las_imp.shape[0]+1))
las_imp['ranking'] = ranking
las_imp

/Users/Lena/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Score for training data: 0.9850992865723173
(not log) RMSE for training data is: 67.43695143644987
Score for test data: 0.9887076999539471
(not log) RMSE for test data is: 64.80593506328668


,Feature,Importance,Abs Importance,ranking
141,log_value,0.375592,0.375592,1
139,interest,-0.005493,0.005493,2
24,median_rent,0.004418,0.004418,3
46,owner_occupied_housing_units_lower_value_quartile,0.004372,0.004372,4
138,gdp,0.003690,0.003690,5
...,...,...,...,...
52,dwellings_1_units_detached,-0.000000,0.000000,148
53,dwellings_1_units_attached,-0.000000,0.000000,149
55,dwellings_3_to_4_units,-0.000000,0.000000,150
56,dwellings_5_to_9_units,0.000000,0.000000,151


In [17]:
r2_train = las.score(c2_strain,Y_c2train)
r2_test = las.score(c2_stest,Y_c2test)
lasso_output['Clust_2'] = [train_RMSE, Test_RMSE, r2_train, r2_test]
lasso_output

,all,all_PCA,Clust_1,Clust_1_PCA,Clust_2
RMSE_train,76.623807,76.576813,90.730051,93.022807,67.436951
RMSE_test,83.510180,86.451560,139.706560,79.819147,64.805935
R2_train,0.987961,0.987916,0.985660,0.985199,0.985099
R2_test,0.986912,0.986288,0.965895,0.987261,0.988708


In [18]:
# Run a PCA on Cluster 2:
scaler_pca = StandardScaler().fit(c2_train)
features = scaler_pca.transform(c2_train)
df2_scal = pd.DataFrame(features, columns = c2_train.columns)

pca = PCA(n_components = 0.95)
df_reduced = pca.fit_transform(df2_scal)

df_reduced = pd.DataFrame(pca.components_, columns = c2_train.columns, index = list(range(1,df_reduced.shape[1]+1)))
df_reduced = df_reduced.apply(lambda x: abs(x))

feat2 = []
for i in range(0,df_reduced.shape[0]):
    ff = df_reduced.iloc[i].idxmax()
    df_reduced.drop(ff, inplace = True, axis = 1)
    feat2.append(ff)

feat2.append('log_value')

c2_train = c2_train[feat2]
c2_test = c2_test[feat2]

# Run Lasso Regression on Cluster 2 with PCA variables:

scaler = StandardScaler().fit(c2_train)
features = scaler.transform(c2_train)
c2_strain = pd.DataFrame(features, columns = c2_train.columns)

alphas = 10**np.linspace(10,-2,500)*0.5

lassocv = LassoCV(alphas = alphas, random_state=0)
lassocv.fit(c2_strain, Y_c2train)
lassocv_alpha = lassocv.alpha_

las = Lasso(alpha = lassocv.alpha_, random_state = 0)
las.fit(c2_strain,Y_c2train)

print('Score for training data:', las.score(c2_strain,Y_c2train))

pred = las.predict(c2_strain)
pred = np.exp(pred)
Y_c2_train = np.exp(Y_c2train)
train_RMSE = mean_squared_error(Y_c2_train, pred, squared = False)
print('(not log) RMSE for training data is:', train_RMSE)

test_features = scaler.transform(c2_test)
c2_stest = pd.DataFrame(test_features, columns = c2_test.columns)

print('Score for test data:', las.score(c2_stest,Y_c2test))

pred_test = las.predict(c2_stest)
pred_test = np.exp(pred_test)
Y_c2_test = np.exp(Y_c2test)
Test_RMSE = mean_squared_error(Y_c2_test, pred_test, squared = False)
print('(not log) RMSE for test data is:', Test_RMSE)

las_imp = pd.DataFrame()

las_imp['Feature'] = c2_strain.columns
las_imp['Importance'] = list(las.coef_)
las_imp['Abs Importance'] = abs(las_imp['Importance'])
las_imp = las_imp.sort_values(by = 'Abs Importance', ascending = False)
las_imp

ranking = list(range(1,las_imp.shape[0]+1))
las_imp['ranking'] = ranking
las_imp

/Users/Lena/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Score for training data: 0.9842592710370169
(not log) RMSE for training data is: 69.43509277865222
Score for test data: 0.9850734001297559
(not log) RMSE for test data is: 73.31188897855804


,Feature,Importance,Abs Importance,ranking
48,log_value,0.382696,0.382696,1
17,gdp,0.004367,0.004367,2
6,cpi,-0.001968,0.001968,3
3,median_income,0.001200,0.001200,4
45,employed_arts_entertainment_recreation_accommo...,0.000620,0.000620,5
26,female_female_households,0.000566,0.000566,6
36,dwellings_2_units,-0.000536,0.000536,7
30,employed_information,0.000405,0.000405,8
15,asian_ratio,0.000396,0.000396,9
7,housing_built_2000_to_2004,0.000037,0.000037,10


In [19]:
r2_train = las.score(c2_strain,Y_c2train)
r2_test = las.score(c2_stest,Y_c2test)
lasso_output['Clust_2_PCA'] = [train_RMSE, Test_RMSE, r2_train, r2_test]
lasso_output

,all,all_PCA,Clust_1,Clust_1_PCA,Clust_2,Clust_2_PCA
RMSE_train,76.623807,76.576813,90.730051,93.022807,67.436951,69.435093
RMSE_test,83.510180,86.451560,139.706560,79.819147,64.805935,73.311889
R2_train,0.987961,0.987916,0.985660,0.985199,0.985099,0.984259
R2_test,0.986912,0.986288,0.965895,0.987261,0.988708,0.985073


In [20]:
# Run Lasso Regression on Cluster 3:

scaler = StandardScaler().fit(c3_train)
features = scaler.transform(c3_train)
c3_strain = pd.DataFrame(features, columns = c3_train.columns)

alphas = 10**np.linspace(10,-2,500)*0.5

lassocv = LassoCV(alphas = alphas, random_state=0)
lassocv.fit(c3_strain, Y_c3train)
lassocv_alpha = lassocv.alpha_

las = Lasso(alpha = lassocv.alpha_, random_state = 0)
las.fit(c3_strain,Y_c3train)

print('Score for training data:', las.score(c3_strain,Y_c3train))

pred = las.predict(c3_strain)
pred = np.exp(pred)
Y_c3_train = np.exp(Y_c3train)
train_RMSE = mean_squared_error(Y_c3_train, pred, squared = False)
print('(not log) RMSE for training data is:', train_RMSE)

test_features = scaler.transform(c3_test)
c3_stest = pd.DataFrame(test_features, columns = c3_test.columns)

print('Score for test data:', las.score(c3_stest,Y_c3test))

pred_test = las.predict(c3_stest)
pred_test = np.exp(pred_test)
Y_c3_test = np.exp(Y_c3test)
Test_RMSE = mean_squared_error(Y_c3_test, pred_test, squared = False)
print('(not log) RMSE for test data is:', Test_RMSE)

las_imp = pd.DataFrame()

las_imp['Feature'] = c3_strain.columns
las_imp['Importance'] = list(las.coef_)
las_imp['Abs Importance'] = abs(las_imp['Importance'])
las_imp = las_imp.sort_values(by = 'Abs Importance', ascending = False)
las_imp

ranking = list(range(1,las_imp.shape[0]+1))
las_imp['ranking'] = ranking
las_imp

/Users/Lena/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Score for training data: 0.9866664980697437
(not log) RMSE for training data is: 68.56087085550124
Score for test data: 0.9862607563300414
(not log) RMSE for test data is: 76.00937481387443


,Feature,Importance,Abs Importance,ranking
141,log_value,0.340581,0.340581,1
139,interest,-0.006346,0.006346,2
138,gdp,0.004741,0.004741,3
24,median_rent,0.003702,0.003702,4
46,owner_occupied_housing_units_lower_value_quartile,0.003374,0.003374,5
...,...,...,...,...
53,dwellings_1_units_attached,0.000000,0.000000,148
54,dwellings_2_units,-0.000000,0.000000,149
55,dwellings_3_to_4_units,-0.000000,0.000000,150
56,dwellings_5_to_9_units,0.000000,0.000000,151


In [21]:
r2_train = las.score(c3_strain,Y_c3train)
r2_test = las.score(c3_stest,Y_c3test)
lasso_output['Clust_3'] = [train_RMSE, Test_RMSE, r2_train, r2_test]
lasso_output

,all,all_PCA,Clust_1,Clust_1_PCA,Clust_2,Clust_2_PCA,Clust_3
RMSE_train,76.623807,76.576813,90.730051,93.022807,67.436951,69.435093,68.560871
RMSE_test,83.510180,86.451560,139.706560,79.819147,64.805935,73.311889,76.009375
R2_train,0.987961,0.987916,0.985660,0.985199,0.985099,0.984259,0.986666
R2_test,0.986912,0.986288,0.965895,0.987261,0.988708,0.985073,0.986261


In [22]:
# Run a PCA on Cluster 3:
scaler_pca = StandardScaler().fit(c3_train)
features = scaler_pca.transform(c3_train)
df3_scal = pd.DataFrame(features, columns = c3_train.columns)

pca = PCA(n_components = 0.95)
df_reduced = pca.fit_transform(df3_scal)

df_reduced = pd.DataFrame(pca.components_, columns = c3_train.columns, index = list(range(1,df_reduced.shape[1]+1)))
df_reduced = df_reduced.apply(lambda x: abs(x))

feat3 = []
for i in range(0,df_reduced.shape[0]):
    ff = df_reduced.iloc[i].idxmax()
    df_reduced.drop(ff, inplace = True, axis = 1)
    feat3.append(ff)

feat3.append('log_value')

c3_train = c3_train[feat3]
c3_test = c3_test[feat3]

# Run Lasso Regression on Cluster 3 with PCA variables:

scaler = StandardScaler().fit(c3_train)
features = scaler.transform(c3_train)
c3_strain = pd.DataFrame(features, columns = c3_train.columns)

alphas = 10**np.linspace(10,-2,500)*0.5

lassocv = LassoCV(alphas = alphas, random_state=0)
lassocv.fit(c3_strain, Y_c3train)
lassocv_alpha = lassocv.alpha_

las = Lasso(alpha = lassocv.alpha_, random_state = 0)
las.fit(c3_strain,Y_c3train)

print('Score for training data:', las.score(c3_strain,Y_c3train))

pred = las.predict(c3_strain)
pred = np.exp(pred)
Y_c3_train = np.exp(Y_c3train)
train_RMSE = mean_squared_error(Y_c3_train, pred, squared = False)
print('(not log) RMSE for training data is:', train_RMSE)

test_features = scaler.transform(c3_test)
c3_stest = pd.DataFrame(test_features, columns = c3_test.columns)

print('Score for test data:', las.score(c3_stest,Y_c3test))

pred_test = las.predict(c3_stest)
pred_test = np.exp(pred_test)
Y_c3_test = np.exp(Y_c3test)
Test_RMSE = mean_squared_error(Y_c3_test, pred_test, squared = False)
print('(not log) RMSE for test data is:', Test_RMSE)

las_imp = pd.DataFrame()

las_imp['Feature'] = c3_strain.columns
las_imp['Importance'] = list(las.coef_)
las_imp['Abs Importance'] = abs(las_imp['Importance'])
las_imp = las_imp.sort_values(by = 'Abs Importance', ascending = False)
las_imp

ranking = list(range(1,las_imp.shape[0]+1))
las_imp['ranking'] = ranking
las_imp

/Users/Lena/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Score for training data: 0.9856827651971015
(not log) RMSE for training data is: 72.24098791792946
Score for test data: 0.9818085833222702
(not log) RMSE for test data is: 89.6842894572063


,Feature,Importance,Abs Importance,ranking
41,log_value,0.345430,0.345430,1
17,gdp,0.005777,0.005777,2
6,cpi,-0.002210,0.002210,3
30,renter_occupied_housing_units_paying_cash_medi...,0.001375,0.001375,4
3,households_retirement_income,-0.000666,0.000666,5
32,housing_built_1939_or_earlier,0.000292,0.000292,6
25,commute_25_29_mins,-0.000000,0.000000,7
26,mobile_homes,-0.000000,0.000000,8
27,less_one_year_college,-0.000000,0.000000,9
28,dwellings_10_to_19_units,0.000000,0.000000,10


In [23]:
r2_train = las.score(c3_strain,Y_c3train)
r2_test = las.score(c3_stest,Y_c3test)
lasso_output['Clust_3_PCA'] = [train_RMSE, Test_RMSE, r2_train, r2_test]
lasso_output

,all,all_PCA,Clust_1,Clust_1_PCA,Clust_2,Clust_2_PCA,Clust_3,Clust_3_PCA
RMSE_train,76.623807,76.576813,90.730051,93.022807,67.436951,69.435093,68.560871,72.240988
RMSE_test,83.510180,86.451560,139.706560,79.819147,64.805935,73.311889,76.009375,89.684289
R2_train,0.987961,0.987916,0.985660,0.985199,0.985099,0.984259,0.986666,0.985683
R2_test,0.986912,0.986288,0.965895,0.987261,0.988708,0.985073,0.986261,0.981809
